# Lambda as py, or a flock of functions

## Combinators, Lambda Calculus, and Church Encodings in JavaScript

### Notes for [a talk by Gabriel Lebec (click for videos and slides)](https://github.com/glebec/lambda-talk)


In [1]:
def header(text):
    print("\n\033[1m\033[4m", text, "\033[0m\n")


def demo(text, boolean):
    from types import FunctionType
    if isinstance(boolean, FunctionType):
        # We have an LC type boolean
        boolean = boolean(True)(False)
    if not isinstance(boolean, bool):
        raise TypeError("Expected a boolean as the second argument")
    mark = "\033[92m✔\033[0m" if boolean else "\033[91m✖\033[0m"
    print(boolean, mark, text)

## Introduction

The Lambda Calculus is a symbol manipulation framework developed by the mathematician Alonzo Church in the 1930s. It was intended to be an extremely tiny syntax which could nevertheless suffice to calculate anything computable. The mathematical advantage of such a syntax is that with such extreme simplicity, it becomes easier to write formal proofs about computational logic – demonstrated when Church solved David Hilbert's famous Decision Problem using LC.

Another famous mathematician, Alan Turing, formulated a different model of universal computation – the eponymous Turing Machine. A TM is a hypothetical device, again of extreme simplicity; it can read or write to cells on an infinite tape, each cell containing data or instructions. Turing published a paper also solving the Decision Problem, mere months after Church.

Turing proved that the Turing Machine and Lambda Calculus are totally equivalent. Everything that one can calculate, the other can. Not only this, but Turing and Church posited (in the "Church-Turing Thesis") that these systems capture the definition of computability in a universal way.

Turing Machines are exciting because if a hypothetical machine can compute anything computable, then perhaps a real machine can as well. Modern computers add many features and optimizations beyond what is featured in an actual Turing Machine; these features make the machine more convenient and performant, but do not compromise its essential nature as a universal computing device.

As machine codes, assemblers, compilers, and higher-level languages have developed to program these real machines, they have largely evolved with a focus on the essence of the machine – memory, statefulness, effects, imperative instructions and so forth. Over time, some of these languages have shifted ever farther into pure abstractions and conceptual description over more machine-centric and stateful models.

However, mathematicians and computer scientists have long known that the entirely abstract lambda calculus, being equivalent to a TM, meant that computations could be expressed in a style totally independent from machine instructions. Instead, a language consisting of first-class function expressions – aka lambda abstractions – could be subsequently compiled into machine code. Such a language would benefit from decades of mathematical research. And just as real computers extend Turing Machines with extra power and convenience, these _functional languages_ would extend the lambda calculus with additional features and under-the-hood shortcuts (such as hardware-based arithmetic).

What follows is a demonstration, mostly for enjoyment and insight, of the surprising and delightful omnipotence of the lambda calculus. Python, like any functional language, is closely related to LC; it contains the necessary ingredients (variables, parentheses, first-class functions) nestled among the additional luxuries most programmers take for granted.

## First steps with simple combinators

Starting simple. A lambda abstraction is simply an anonymous unary function. In LC there are no named functions, but for convenience we will use names. In mathematical notation, `:=` means "is defined as".


## Identity

Here is an almost trivial function: Identity, defined as `λx.x`. `λ` is used to indicate the start of a lambda abstraction (read: function expression). The sole parameter is listed to the left of the `.`, and the return expression is written after the `.`

In [2]:
header("Idiot := I := λx.x")
I = lambda x: x


 Idiot := I := λx.x 



In LC, variables may be static tokens that stand for whatever you want. We'll use python strings for this.

In [3]:
tweet = "tweet"
chirp = "chirp"

In LC, juxtaposition is function application. That is, `M I` means "apply the `M` function to the `I` argument". A space is not strictly necessary; `fx` usually means "apply `f` to `x`". Some of our examples will use multi-letter variables; to disambiguate them from multiple single-letter variables, we will frequently use SPACED UPPERCASE.
```
I tweet
= (λx.x) tweet
= tweet
```

In [4]:
demo('I tweet = tweet', I(tweet) == tweet)
demo('I chirp = chirp', I(chirp) == chirp)

True ✔ I tweet = tweet
True ✔ I chirp = chirp


In LC, valid arguments include other lambda abstractions. This is the meaning of "first-class" in the phrase "first-class functions"; functions are values which may be passed into or returned from other functions. In this sense, functions are both like verbs (they can perform a calculation) and nouns (they can be the subject or object of a calculation).

```
I I
= (λx.x)(λx.x)
= λx.x
```


In [5]:
demo('I I = I', I(I) == I)

True ✔ I I = I


Of course we can build up more complex expressions. In LC, function application associates left; that is, `a b c d` = `((a b) c) d`. Evaluating terms by substituting arguments into function bodies has the fancy term of "β-reduction". A reducible expression (or "redex") that has been completely simplified in this fashion is said to be in its "β-normal" or just "normal" form.

```
     id    id       id    id  tweet   <--- the initial reducible expression ("redex")
= (((id    id)      id)   id) tweet   <--- application associates left
= ((    id          id)   id) tweet   <--- evaluating function applications ("β-reduction")
= (           id          id) tweet   <--- continuing β-reduction
=                    id       tweet   <--- more β-reduction
=                             tweet   <--- normal form of the original redex
```

LC has great overlap with combinatory logic. A pioneer of CL was the mathematician Haskell Curry, whose name now adorns the Haskell language as well as the functional concept of currying. (Curry actually cited the technique from Schönfinkel, and Frege used it even earlier.) Combinatory logic is concerned with combinators, that is, functions which operate only on their inputs. The Identity function is a combinator, often abbreviated I, sometimes called the Idiot bird.


In [6]:
header('Idiot := I')
Idiot = I


 Idiot := I 



## Self-Application

Curry was an avid birdwatcher, and the logician Raymond Smullyan named many combinators after birds in his honor. The self-application combinator M (for "Mockingbird"), aka ω (lowercase omega), looks like this:


In [7]:
header("Mockingbird := M := ω := λf.ff")


 Mockingbird := M := ω := λf.ff 



Remember, `fx` means "apply f to x." So `ff` means "apply f to itself".

In [8]:
ω = lambda fn: fn(fn)
M = ω
Mockingbird = M

Can you guess what the Mockingbird of Identity is?

```
M I
= (λf.ff)(λx.x)
= (λx.x)(λx.x) = I I
= λx.x = I
```

That's right, it's the same as Identity of Identity… which is Identity.

In [9]:
demo('M I = I I = I', M(I) == I(I) == I)

True ✔ M I = I I = I


What about… the Mockingbird of Mockingbird?

That's the Ω (big Omega) Combinator. It diverges (goes on forever).

```
M M
= (λf.ff)(λf.ff)
= (λf.ff)(λf.ff)
= (λf.ff)(λf.ff)
= (λf.ff)(λf.ff)
...
```

In [10]:
try:
    M(M)
except RecursionError as ex:
    print(ex)

maximum recursion depth exceeded


## Church Encodings: Booleans

In [11]:
header('First encodings: boolean value functions')


 First encodings: boolean value functions 



OK this is nice and all, but how can we do anything real with only functions? Let's start with booleans. Here we need some multi-arg functions… in lambda calc, `λabc.a` is just shorthand for `λa.λb.λc.a`, or `λa.(λb.(λc.a)))`. In other words, all functions are curried.

### True and False


In [12]:
header("T := λxy.x")
T = lambda thn: lambda els: thn


 T := λxy.x 



In [13]:
header("F := λxy.y")
F = lambda thn: lambda els: els


 F := λxy.y 



Hm. How can "true" and "false" be functions? Well, the point of booleans is to _select_ between a then-case and an else-case. The LC booleans are just functions which do that! `T` takes two vals and returns the first; `F` takes two vals and returns the second. If you apply an unknown bool func to two vals, the first val will be returned if the bool was true, else the second val will be returned.

In [14]:
demo('T tweet chirp = tweet', T(tweet)(chirp) == tweet)
demo('F tweet chirp = chirp', F(tweet)(chirp) == chirp)

True ✔ T tweet chirp = tweet
True ✔ F tweet chirp = chirp


### Flipping Arguments

Another fun way we could have produced F was with the Cardinal combinator. The Cardinal, aka `C`, aka `flip`, takes a binary (two-argument) function, and produces a function with reversed argument order.

In [15]:
header("Cardinal := C := flip := λfab.fba")
flip = lambda func: lambda a: lambda b: func(b)(a)
C = flip
Cardinal = C


 Cardinal := C := flip := λfab.fba 



With the Cardinal, we can derive `F` from the flip of `T`:

In [16]:
header("F = C T")


 F = C T 



In [17]:
demo('flip T tweet chirp = chirp', flip(T)(tweet)(chirp) == chirp)

True ✔ flip T tweet chirp = chirp


Regardless of whether you define `F` manually, or as the flip of `T`, these functions are _encodings_ of boolean values. They represent booleans in useful and meaningful ways, which preserve the behavior of the values. Specifically, they are Church encodings – representations developed / discovered by Alonzo Church. Their real power is revealed when we start defining logical operations on them.

### Negation

In [18]:
header("NOT := λb.bFT")
NOT = lambda chooseOne: chooseOne(F)(T)


 NOT := λb.bFT 



Remember, booleans in the LC are really functions which select one of two arguments. Try reducing the expression `NOT T` yourself, then check your work below.
```
NOT T
= (λb.bFT) T
= TFT
= (λxy.x) F T
= F
```

In [19]:
demo('NOT T = F', NOT(T) == F)
demo('NOT F = T', NOT(F) == T)

True ✔ NOT T = F
True ✔ NOT F = T


There's another way we could define `NOT`, however, and we've already seen it. Remember how the flip of `T` (aka the Cardinal of `T`) is `F`? It works the other way too – `C F = T`! At least, `C F` yields a function that behaves identically to `T`:

In [20]:
demo('CF = T', C(F)(tweet)(chirp) == tweet)
demo('CT = F', C(T)(tweet)(chirp) == chirp)

True ✔ CF = T
True ✔ CT = F


This means that when it comes to Church encodings of booleans, the Cardinal behaves just like our manually-defined `NOT`.

### Conjunction and Disjunction

Can you construct a function for boolean `AND`? It will have to take two unknown boolean functions as parameters, and route to an output of the correct boolean result. Give it a try.

In [21]:
header("AND := λpq.pqF")


 AND := λpq.pqF 



(λpq.pqp also works; can you see why?)

In [22]:
AND = lambda p: lambda q: p(q)(F)

In [23]:
demo('AND F F = F', AND(F)(F) == F)
demo('AND T F = F', AND(T)(F) == F)
demo('AND F T = F', AND(F)(T) == F)
demo('AND T T = T', AND(T)(T) == T)

True ✔ AND F F = F
True ✔ AND T F = F
True ✔ AND F T = F
True ✔ AND T T = T


If you work through the logic, you might notice that this function exhibits short-circuiting. If `p = F`, we don't bother using `q`. Similarly, our `OR` function below short circuits; if `p = T`, we don't bother using `q`.

In [24]:
header("OR := λpq.pTq")


 OR := λpq.pTq 



(λpq.ppq also works)

In [25]:
OR = lambda p: lambda q: p(T)(q)

In [26]:
demo('OR F F = F', OR(F)(F) == F)
demo('OR T F = T', OR(T)(F) == T)
demo('OR F T = T', OR(F)(T) == T)
demo('OR T T = T', OR(T)(T) == T)

True ✔ OR F F = F
True ✔ OR T F = T
True ✔ OR F T = T
True ✔ OR T T = T


`λpq.ppq` also works because if `p` is true, it is supposed to select `T`, but `p = T`, so we can just reuse it. Notice something interesting here: `λpq.ppq` behaves exactly like the Mockingbird. It takes a value, `p`, and self-applies `p` (producing `pp`). Well, `Mp = pp`, and you can apply that result to another value `q`, to get `ppq`; therefore, `Mpq = ppq`. So `M` and `λpq.ppq` behave identically (i.e. they are "extensionally equivalent", and in fact `λpq.ppq` is sometimes called `M*` or "the Mockinbird once-removed". The Mockingbird works as a boolean `OR` function:

In [27]:
demo('M F F = F', M(F)(F) == F)
demo('M T F = T', M(T)(F) == T)
demo('M F T = T', M(F)(T) == T)
demo('M T T = T', M(T)(T) == T)

True ✔ M F F = F
True ✔ M T F = T
True ✔ M F T = T
True ✔ M T T = T


### Demo: De Morgan's Laws

With working booleans, we can illustrate some more complex logic, such as one of De Morgan's Laws: `!(p && q) === (!p) || (!q)`.

In [28]:
header("De Morgan: not (and P Q) = or (not P) (not Q)")


 De Morgan: not (and P Q) = or (not P) (not Q) 



In [29]:
def deMorgansLawDemo(p, q):
    return NOT(AND(p)(q)) == OR(NOT(p))(NOT(q))

In [30]:
demo("NOT (AND F F) = OR (NOT F) (NOT F)", deMorgansLawDemo(F, F))
demo("NOT (AND T F) = OR (NOT T) (NOT F)", deMorgansLawDemo(T, F))
demo("NOT (AND F T) = OR (NOT F) (NOT T)", deMorgansLawDemo(F, T))
demo("NOT (AND T T) = OR (NOT T) (NOT T)", deMorgansLawDemo(T, T))

True ✔ NOT (AND F F) = OR (NOT F) (NOT F)
True ✔ NOT (AND T F) = OR (NOT T) (NOT F)
True ✔ NOT (AND F T) = OR (NOT F) (NOT T)
True ✔ NOT (AND T T) = OR (NOT T) (NOT T)


### Boolean Equality

However, this whole time we have been cheating in the demonstrations. Lambda calculus doesn't have any `===` operator to check for equality! Don't fret though, everything is functions. We can define our own equality function for booleans.


In [31]:
header("BEQ := λpq.p (qTF) (qFT)")
BEQ = lambda p: lambda q: p(q(T)(F))(q(F)(T))


 BEQ := λpq.p (qTF) (qFT) 



In [32]:
demo("BEQ F F = T: ", BEQ(BEQ(F)(F))(T))
demo("BEQ F T = F: ", BEQ(BEQ(F)(T))(F))
demo("BEQ T F = F: ", BEQ(BEQ(T)(F))(F))
demo("BEQ T T = T: ", BEQ(BEQ(T)(T))(T))

True ✔ BEQ F F = T: 
True ✔ BEQ F T = F: 
True ✔ BEQ T F = F: 
True ✔ BEQ T T = T: 


Not a Python operator in sight (our `demo` function accepts church encodings). But for clarity's sake, we'll go back to using Python's equality operator in our `demo` calls.


## Church Encodings: Numerals

In [33]:
header("Numbers")


 Numbers 



Booleans are neat, but surely to compute arithmetic you need language-supported math. Right? …Nah. You can construct math from scratch.

The Church encoding for a natural number n is an n-fold "compositor" (composing combinator). In other words, "2" is a function that composes a function `f` twice: `2 f x = f(f(x))`. Whereas "4" is a function that composes any `f` four times: `4 f x = f(f(f(f(x)`. In this sense, 2 and 3 can be read more like "two-fold" and "three-fold", or "twice" and "thrice".

### Hard-Coded Numbers

In this system, `ZERO` is a function which applies a function `f` zero times to `x`. So… `ZERO` ignores the function argument, just returning `x`; `0 f x = x`.


In [34]:
header("0 := λfx.x")


 0 := λfx.x 



(fun fact, `0 = F`)

In [35]:
ZERO = lambda fn: lambda x: x

Zero applications of Mockingbird to `tweet` is just `tweet`. Good thing too, because applying Mockingbird to `tweet` would throw an error in JS (though it would work fine in LC, just by not simplifying further).

In [36]:
demo("0 M tweet = tweet", ZERO(M)(tweet) == tweet)

True ✔ 0 M tweet = tweet


We could hard-code `ONCE` as a single application of `f`…

In [37]:
header("hard-coded 1 and 2")
header("1 := λfx.fx")


 hard-coded 1 and 2 


 1 := λfx.fx 



In [38]:
ONCE = lambda fn: lambda x: fn(x)

Another fun fact – this is one step removed from `I`, called `I*` ("I-star"). In fact we could have shortened this to be `1 := I`. So 0 is false and 1 is identity, how nice!

In [39]:
demo("1 I tweet = tweet", ONCE(I)(tweet) == tweet)

True ✔ 1 I tweet = tweet


Identity is a bit boring, however, because the n-fold composition of `I` is always `I`, even for n = 0. The above example doesn't really prove our function is doing anything interesting. Let's cheat a bit with some string ops (note, this is polluting LC with some Python, but it's just for demonstration):


In [40]:
λ = "λ"
yell = lambda s: str(s) + "!"

In [41]:
demo("0 yell λ = λ", ZERO(yell)(λ) == "λ")
demo("1 yell λ = yell λ = λ!", ONCE(yell)(λ) == "λ!")

True ✔ 0 yell λ = λ
True ✔ 1 yell λ = yell λ = λ!


we could also hard-code `TWICE`

In [42]:
header("2 := λfx.f(fx)")
TWICE = lambda fn: lambda x: fn(fn(x))


 2 := λfx.f(fx) 



In [43]:
demo("2 yell λ = yell (yell λ) = λ!!", TWICE(yell)(λ) == "λ!!")

True ✔ 2 yell λ = yell (yell λ) = λ!!


### Successor

This hard-coding works, but is very limiting. We can't do true arithmetic like this, where operations on numbers generate other numbers. What we need is a way to count up.

In [44]:
header("SUCCESSOR")


 SUCCESSOR 



In [45]:
header("SUCCESSOR := λfnx.f(nfx)")
SUCCESSOR = lambda num: lambda fn: lambda x: fn(num(fn)(x))


 SUCCESSOR := λfnx.f(nfx) 



Don't get lost in the weeds. All we are saying is that the successor of `n` does `n` compositions of `f` to a value `x`, and then it does _one more_ application of `f` to the result. Therefore, it ends up doing 1 + n compositions of `f` in total.

In [46]:
newOnce = SUCCESSOR(ZERO)
newTwice = SUCCESSOR(SUCCESSOR(ZERO))  # we can use multiple successors on zero, or…
newThrice = SUCCESSOR(newTwice)  # …apply successor to already-obtained numbers.

In [47]:
demo("1 yell λ = λ!", newOnce(yell)(λ) == "λ!")
demo("2 yell λ = λ!!", newTwice(yell)(λ) == "λ!!")
demo("3 yell λ = λ!!!", newThrice(yell)(λ) == "λ!!!")

True ✔ 1 yell λ = λ!
True ✔ 2 yell λ = λ!!
True ✔ 3 yell λ = λ!!!


### Composition and Point-Free Notation

There is another way to write successor. Point-free (some joke "point-less") notation means to define a function purely as a combination of other functions, without explicitly writing final arguments. Sometimes this style reveals what a function *is* rather than what explain what it *does*. Other times it can be abused to produce incomprehensible gibberish. Successor is a reasonable candidate for it, however.

We are doing n-fold compositions, so let's define an actual `compose` function to help. Composition is often notated as `∘` in infix position: `(f ∘ g) x = f(g(x))`. However, Lambda Calculus only includes prefix position function application. Smullyan named this the Bluebird after Curry's `B` combinator.

In [48]:
header("Bluebird := B := (∘) := compose := λfgx.f(gx)")
compose = lambda f: lambda g: lambda x: f(g(x))
B = compose
Bluebird = B


 Bluebird := B := (∘) := compose := λfgx.f(gx) 



In [49]:
demo("(B NOT NOT)  T =  NOT (NOT T)", (B(NOT)(NOT))(T) == NOT(NOT(T)) == T)
demo("(B yell NOT) F = yell (NOT F)", (B(yell)(NOT))(F) == yell(NOT(F)) == str(T) + "!")

True ✔ (B NOT NOT)  T =  NOT (NOT T)
True ✔ (B yell NOT) F = yell (NOT F)


Now that we have an actual composition function, we can define successor without mentioning the final `x` value argument.

In [50]:
header("SUCC := λnf.f∘(nf) = λnf.Bf(nf)")
SUCC = lambda num: lambda fn: compose(fn)(num(fn))


 SUCC := λnf.f∘(nf) = λnf.Bf(nf) 



This is just a terse way of repeating what we already know: if a given `n` composes some function `f` n times, then the successor of n is a function which composes one additional `f`, for a total of 1 + n compositions.

In [51]:
n0 = ZERO
n1 = SUCC(n0)
n2 = SUCC(SUCC(n0))
n3 = SUCC(SUCC(SUCC(n0)))
n4 = SUCC(n3)

In [52]:
demo("1 yell λ = λ!", n1(yell)(λ) == "λ!")
demo("2 yell λ = λ!!", n2(yell)(λ) == "λ!!")
demo("3 yell λ = λ!!!", n3(yell)(λ) == "λ!!!")
demo("4 yell λ = λ!!!!", n4(yell)(λ) == "λ!!!!")

True ✔ 1 yell λ = λ!
True ✔ 2 yell λ = λ!!
True ✔ 3 yell λ = λ!!!
True ✔ 4 yell λ = λ!!!!


### Arithemtic

#### Addition

Things will get pretty slow if we can only increment by 1. Let's add addition.

In [53]:
header("ADD := λab.a(succ)b")
ADD = lambda numA: lambda numB: numA(SUCC)(numB)


 ADD := λab.a(succ)b 



Aha, addition is just the Ath successor of B. Makes sense. For example, `ADD 3 2 = 3 SUCC 2`, which could be read as "thrice successor of twice".

In [54]:
n5 = ADD(n2)(n3)
n6 = ADD(n3)(n3)

In [55]:
demo("ADD 5 2 yell λ = λ!!!!!!!", ADD(n5)(n2)(yell)(λ) == "λ!!!!!!!")
demo("ADD 0 3 yell λ = λ!!!", ADD(n0)(n3)(yell)(λ) == "λ!!!")
demo("ADD 2 2 = 4", ADD(n2)(n2)(yell)(λ) == n4(yell)(λ))

True ✔ ADD 5 2 yell λ = λ!!!!!!!
True ✔ ADD 0 3 yell λ = λ!!!
True ✔ ADD 2 2 = 4


These equivalence checks using `yell` and `'λ'` are a bit verbose. It's annoying, but a mathematical truth, that there can be no general algorithm to decide if two functions are equivalent – and we cannot rely on Python function equality because we are generating independent function objects. Since `yell` and `'λ'` are already impure non-LC code, we might as well go all the way and define `church` and `pynum` to convert between Church encodings and Python numbers.

In [56]:
header("LC <-> Python: church & pynum")


 LC <-> Python: church & pynum 



In [57]:
def church(n: int):
    """Convert python integer to Church encodings"""
    return n0 if n == 0 else SUCC(church(n - 1))

In [58]:
def pynum(c):
    """Convert Curch encodings to python integer"""
    return c(lambda x: x + 1)(0)

In [59]:
demo("church(5) = n5", church(5)(yell)(λ) == n5(yell)(λ))
demo("pynum(n5) === 5", pynum(n5) == 5)
demo("pynum(church(2)) === 2", pynum(church(2)) == 2)

True ✔ church(5) = n5
True ✔ pynum(n5) === 5
True ✔ pynum(church(2)) === 2


#### Multiplication

Back to math. How about multiplication?

In [60]:
header("MULT := λab.a∘b = compose")
MULT = compose


 MULT := λab.a∘b = compose 



How beautiful! Multiplication is the composition of numbers. For example, `MULT 3 2 = 3 ∘ 2`, which could be read as "thrice of twice", or "three of (two of (someFn))". If you compose a function `f` two times — `f ∘ f` — and then you compose that result three times — `(f∘f) ∘ (f∘f) ∘ (f∘f)` — you get the six-fold composition of f: `f ∘ f ∘ f ∘ f ∘ f ∘ f`. It helps to know that composition is associative — `f ∘ (g ∘ h) = (f ∘ g) ∘ h`.

In [61]:
demo("MULT 1 5 = 5", pynum(MULT(n1)(n5)) == 5)
demo("MULT 3 2 = 6", pynum(MULT(n3)(n2)) == 6)
demo("MULT 4 0 = 0", pynum(MULT(n4)(n0)) == 0)
demo("MULT 6 2 yell λ = λ!!!!!!!!!!!!", MULT(n6)(n2)(yell)(λ) == "λ!!!!!!!!!!!!")

True ✔ MULT 1 5 = 5
True ✔ MULT 3 2 = 6
True ✔ MULT 4 0 = 0
True ✔ MULT 6 2 yell λ = λ!!!!!!!!!!!!


In [62]:
n8 = MULT(n4)(n2)
n9 = SUCC(n8)

#### Exponentiation

Exponentiation is remarkably clean too. When we say 2^3, we are saying "multiply two by itself three times"; or putting it another way, "twice of twice of twice". So for any base and power, the result is the power-fold composition of the base:

In [63]:
header("Thrush := POW := λab.ba")
POW = lambda numA: lambda numB: numB(numA)


 Thrush := POW := λab.ba 



As you can see, we also call this combinator the Thrush. Unfortunately we have a name collision as we already defined `T` as the church encoding of true, so we omit the single-letter version of this combinator. There is another letter sometimes reserved for true, in which case we can use `T` for Thrush; the alternate true combinator name is coming up soon.